# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
# !pip install opencv-contrib-python
# !pip3 install scipy==1.2
# !pip3 install livelossplot


In [ ]:
### Importing required packages
import numpy as np
import os
# from scipy.misc import imread, imresize
import cv2
import datetime
import warnings
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
# !pip3 install livelossplot 
from livelossplot import PlotLossesKeras
import random as rn
import tensorflow as tf
tf.enable_v2_behavior()
warnings.filterwarnings("ignore")

In [ ]:
# freezing the values
np.random.seed(30)
rn.seed(30)
# tf.random.set_seed(30)

In [ ]:
# importing the csv file comtaining the meta data for the images
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
# Setting image path
train_path = 'Project_data/train'
val_path = 'Project_data/val'

curr_dt_time = datetime.datetime.now()

In [ ]:
batch_size = 64
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = num_train_sequences//batch_size# choose the number of epochs
print ('# epochs =', num_epochs)

# The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide
# the number of next() calls it need to make.

steps_per_epoch = int(num_train_sequences/batch_size) if (num_train_sequences%batch_size) == 0 else (num_train_sequences//batch_size) + 1
validation_steps = int(num_val_sequences/batch_size) if (num_val_sequences%batch_size) == 0 else (num_val_sequences//batch_size) + 1
    
print("# steps_per_epoch:", steps_per_epoch)
print("# validation_steps:", validation_steps)


In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch', period=4)

LR = ReduceLROnPlateau()

early = EarlyStopping(patience=4)

callbacks_list = [LR]

In [ ]:
def plot(history):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
    axes[0].plot(history.history['loss'])   
    axes[0].plot(history.history['val_loss'])
    axes[0].legend(['loss','val_loss'])
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    axes[1].plot(history.history['categorical_accuracy'])   
    axes[1].plot(history.history['val_categorical_accuracy'])
    axes[1].legend(['categorical_accuracy','val_categorical_accuracy'])
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    
    plt.show()

In [ ]:
import pandas as pd
data = pd.read_csv("Project_data/train.csv", header=None)
data.columns = ["temp"]
data['temp'].str.split(";", expand = True)

In [ ]:
def generator(source_path, folder_list, batch_size, x=20, y=180, z=180):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [i for i in range(0,30,2)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
#                     #crop the images and resize them. Note that the images are of 2 different shape 
#                     #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[1] == 160:
                        image = cv2.resize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = cv2.resize(image,(84,84)).astype(np.float32)

#                     #Converting to gray scale
#                     temp = temp.mean(axis=-1,keepdims=1) 
#                     temp = temp/127.5-1 #Normalize data
#                     batch_data[folder,idx] = temp #normalise and feed in the image
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123                    
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[1] == 160:
                        image = cv2.resize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = cv2.resize(image,(84,84)).astype(np.float32)
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123                    

                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
input_shape= (18,84,84,3)

# mod = VGG16(weights = "imagenet", include_top=False)

from tensorflow.python.keras.layers import Input, Dense, TimeDistributed, MaxPooling3D, Conv3D


# from tf.layers import TimeDistributed, MaxPooling2D, Conv2D

def model_1(input_shape = input_shape):
    # Define model1
    model1 = Sequential()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
    # model1.add(TimeDistributed(mod,input_shape=(224, 224, 3)))
    model1.add(Conv3D(32, kernel_size=(3,3,3), input_shape=input_shape,padding='same', activation = 'elu', strides=(1,1,1)))
    model1.add(BatchNormalization())
    model1.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

    model1.add(Conv3D(64, kernel_size=(3,3,3),padding='same', activation = 'elu', strides=(1,1,1)))
    model1.add(BatchNormalization())
    model1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

    model1.add(Conv3D(128, kernel_size=(3,3,3),padding='same', activation = 'elu', strides=(1,1,1)))
    model1.add(BatchNormalization())
    model1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
     
    #Flatten Layers
    model1.add(Flatten())
    model1.add(Dropout(0.5))
    model1.add(Dense(256, activation='elu'))
    model1.add(Dropout(0.5))
    
    #softmax layer
    model1.add(Dense(5, activation='softmax'))
    return model1

In [ ]:
optimiser = optimizers.SGD(0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
model_test1 = model_1(input_shape= input_shape)
model_test1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
batch_size = 32
train_generator = generator(train_path, train_doc, batch_size, x=18, y=84, z=84)
val_generator = generator(val_path, val_doc, batch_size, x=18, y=84, z=84)

In [ ]:
history = model_test1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1, validation_data=val_generator, validation_steps=validation_steps, class_weight=None, workers=-1, initial_epoch=0, callbacks=callbacks_list)

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,31)
plt.subplot(1,2,1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

acc_train = history.history['categorical_accuracy']
acc_val = history.history['val_categorical_accuracy']
plt.subplot(1,2,2)
plt.plot(epochs, acc_train, 'g', label='Training acc')
plt.plot(epochs, acc_val, 'b', label='validation acc')
plt.title('Training and Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()
plt.show()

In [ ]:
loss, accuracy = model_test1.evaluate_generator(val_generator, steps=validation_steps, verbose=1)
print()
print("CLASSIFICATION ACCCURACY")
print()
print("# loss:",loss)
print("# accuracy:",accuracy)

# Using Sequence model

In [ ]:
from tensorflow import keras 
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import TimeDistributed, LSTM, MaxPooling2D

m_net = VGG16(weights='imagenet', include_top=False)

for i in m_net.layers:
    i.trainable = False
    

model = Sequential()
model.add(TimeDistributed(m_net, input_shape = (18, 84, 84, 3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(GRU(128), return_sequence = True)
model.add(Dropout(0.1))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(5, activation='softmax'))


optimiser = optimizers.SGD(0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.summary()

In [ ]:
# callbacks_list = [checkpoint, LR]
model_seq = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1, callbacks=callbacks_list, validation_data=val_generator, validation_steps=validation_steps, class_weight=None, workers=-1, initial_epoch=0)

In [ ]:
loss_train = model_seq.history['loss']
loss_val = model_seq.history['val_loss']
epochs = range(1,31)
plt.subplot(1,2,1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

acc_train = model_seq.history['categorical_accuracy']
acc_val = model_seq.history['val_categorical_accuracy']
plt.subplot(1,2,2)
plt.plot(epochs, acc_train, 'g', label='Training acc')
plt.plot(epochs, acc_val, 'b', label='validation acc')
plt.title('Training and Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model)